In [18]:
import joblib
import pandas as pd
import numpy as np

In [46]:
df1 = pd.read_csv('transactions_1.csv')
df2 = pd.read_csv('transactions_2.csv')

# Concatenating two datasets
df = pd.concat ([df1,df2], axis=0, ignore_index=True).drop(['Unnamed: 0'], axis=1)
# Converting date column to standard format
df['date']= pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
# Filter on one customer
sales = df[df['customer_id'] == 8649525]

# monthly sales to customer "8649525"
sales['date'] = pd.to_datetime(sales['date'])

#represent month in date field as its first day
sales['date'] = sales['date'].dt.year.astype('str') + '-' + sales['date'].dt.month.astype('str') + '-01'
sales['date'] = pd.to_datetime(sales['date'])
#groupby date and sum the sales
sales = sales.groupby('date').size().reset_index()
sales.columns = ['date', 'order_count']
y = sales.set_index('date')

C:\Users\20190041\AppData\Local\Continuum\anaconda3\envs\tensorflow-sessions\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\20190041\AppData\Local\Continuum\anaconda3\envs\tensorflow-sessions\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\20190041\AppData\Local\Continuum\anaconda3\envs\tensorflow-sessions\lib\site-packages\ipykernel_launcher.py:16

In [53]:
# load the model from disk
loaded_model = joblib.load('finalized_model.sav')
pred = loaded_model.get_prediction(start=pd.to_datetime('2019-01-01'), dynamic=False)
result= np.round(pred.predicted_mean, decimals=0)
print(result)

2019-01-01     969.0
2019-02-01    1025.0
2019-03-01    1226.0
2019-04-01     726.0
2019-05-01     648.0
2019-06-01    1525.0
2019-07-01    1049.0
2019-08-01    1362.0
2019-09-01     268.0
2019-10-01     685.0
2019-11-01     207.0
2019-12-01     253.0
2020-01-01     676.0
2020-02-01     335.0
2020-03-01     375.0
Freq: MS, Name: predicted_mean, dtype: float64


In [54]:
# We used RMSE as the metric to measure the performance of the model

y_forecasted = pred.predicted_mean
y_truth = y['order_count']['2019-01-01':]
mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error is {}'.format(round(mse, 2)))
print('The Root Mean Squared Error is {}'.format(round(np.sqrt(mse), 2)))

The Mean Squared Error is 106912.87
The Root Mean Squared Error is 326.98
